# Contextual compression

In [1]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## vector store retriever


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

documents = TextLoader("data/sample.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="snowflake-arctic-embed:33m")
retriever = FAISS.from_documents(texts, embeddings).as_retriever()
docs = retriever.invoke("What did the president say about Ketanji Brown Jackson")
pretty_print_docs(docs)

Document 1:

Cats are known for their quirky personalities and playful antics. They can go from being aloof to affectionate in a heartbeat, keeping their owners on their toes. Whether it’s chasing a laser pointer or pouncing on a stray piece of string, their energy is contagious.

Grooming is a big part of a cat’s day. They spend hours licking their fur, which helps keep it clean and free of parasites. This self-care routine also helps them regulate their body temperature and can be quite soothing.

When it comes to communication, cats have a unique way of expressing themselves. From soft purrs to loud meows, each sound can convey different emotions. A slow blink from a cat is often seen as a sign of trust and affection.

Cats are also known for their love of napping. They can sleep anywhere from 12 to 16 hours a day, making them expert loungers. Finding a sunny spot or a cozy blanket is usually their go-to for a good snooze.


## contextual compression with an `LLMChainExtractor`

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

llm = ChatOllama(model="llama3.2:1b")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

*President*
Ketanji Jackson Brown


## More built-in compressors: filters
### `LLMChainFilter`

In [10]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

### `LLMListwiseRerank`


In [12]:
from langchain.retrievers.document_compressors import LLMListwiseRerank

llm = ChatOllama(model="llama3.2:1b")

_filter = LLMListwiseRerank.from_llm(llm, top_n=1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Brown Jackson"
)
pretty_print_docs(compressed_docs)

ValidationError: 1 validation error for RankDocuments
ranked_document_ids
  Input should be a valid list [type=list_type, input_value='[0, 1, 2]', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/list_type

### `EmbeddingsFilter`


In [13]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

## Adding compressors and document transformers together

In [14]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [15]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)